In [ ]:
#Imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score, mean_squared_error
import joblib
import os

In [ ]:
#Load & Pre-process Data

# Define column names (dataset has no header)
columns = [
    'Age', 'Hours_Studied', 'Parental_Involvement', 'Access_to_Resources',
    'Extracurricular_Activities', 'Sleep_Hours', 'Previous_Scores',
    'Motivation_Level', 'Internet_Access', 'Tutoring_Sessions',
    'Family_Income', 'Teacher_Quality', 'School_Type', 'Peer_Influence',
    'Physical_Activity', 'Learning_Disabilities', 'Parental_Education_Level',
    'Distance_from_Home', 'Gender', 'Exam_Score'
]

# Load data
data = pd.read_csv("data/StudentPerformanceFactors.csv", header=None)
data.columns = columns

# Mapping dictionaries
lowmed_map = {'Low': 0, 'Medium': 1, 'High': 2}
yes_no_map = {'Yes': 1, 'No': 0}
school_type_map = {'Public': 0, 'Private': 1}
influence_map = {'Positive': 1, 'Negative': -1, 'Neutral': 0}
education_map = {'None': 0, 'Primary': 1, 'High School': 2, 'College': 3, 'Postgraduate': 4}
distance_map = {'Near': 0, 'Moderate': 1, 'Far': 2}
gender_map = {'Female': 1, 'Male': 0}

# Fill missing values
data['Teacher_Quality'] = data['Teacher_Quality'].fillna(data['Teacher_Quality'].mode()[0])
data['Parental_Education_Level'] = data['Parental_Education_Level'].fillna(data['Parental_Education_Level'].mode()[0])

# Apply mappings
mapping_cols = {
    'Parental_Involvement': lowmed_map,
    'Access_to_Resources': lowmed_map,
    'Extracurricular_Activities': yes_no_map,
    'Motivation_Level': lowmed_map,
    'Internet_Access': yes_no_map,
    'Family_Income': lowmed_map,
    'Teacher_Quality': lowmed_map,
    'School_Type': school_type_map,
    'Peer_Influence': influence_map,
    'Learning_Disabilities': yes_no_map,
    'Parental_Education_Level': education_map,
    'Distance_from_Home': distance_map,
    'Gender': gender_map
}

for col, mapping in mapping_cols.items():
    data[col] = data[col].map(mapping)

# Drop any remaining NaN
data.dropna(inplace=True)

In [ ]:
# Prepare Features and Labels
X = data.drop('Exam_Score', axis=1).values
y = data['Exam_Score'].values.reshape(-1, 1)

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Define Model
class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

# Initialize
input_dim = X.shape[1]
model = ANNModel(input_dim=input_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train Model
epochs = 1000
for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}")

In [ ]:
# Evaluate & Save
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy()

print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Save
os.makedirs("model", exist_ok=True)
torch.save(model.state_dict(), "model/Ann_exam_score_model.pkl")
joblib.dump(scaler, "model/scaler.pkl")
print("✅ Model and scaler saved!")